In [5]:
import numpy as np
import cv2
import time

import numpy as np
import soundfile as sf
import sounddevice as sd
import ipywidgets as wg
from IPython.display import display

# define a video capture object 
vid = cv2.VideoCapture(0) 
time.sleep(3)

#high speed, low accuraccy 
#tracker = cv2.TrackerMOSSE_create()

#bit slower, more accurate
tracker = cv2.TrackerCSRT_create()

flag = 1

#starts loop
ret, frame = vid.read() 

bbox = cv2.selectROI("Tracking",frame, False)
tracker.init(frame,bbox)

def drawBox(frame,bbox):
    global x_prev, y_prev
    global flag
    x_temp = 0
    y_temp = 0
    
    x, y, w, h = int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]) 
    cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (255,0,255), 3,1)
    cv2.putText(frame,"Tracking",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0),2)

     #first frame, no movement
    if flag == 1:
        x_temp = x
        y_temp = y
        flag = 0
    else:
         #save value of x and y prev
        x_temp = x_prev
        y_temp = y_prev
    
    x_prev = x
    y_prev = y
    
    return x,x_temp,y,y_temp

#initialize synth stream 

def update_amp(y_mov,y_prv):
    if(y_mov < 0):
        y_mov = 0
    if(y_prv < 0):
        y_prv = 0
    print("previous y: "+ str(y_prv))
    print("current y: "+ str(y_mov))
    
    #minimum movment to change parameter
    if(abs(y_prv-y_mov)>= 5):
        print("changing amplitude from "+str(y_prv)+" to"+str(y_mov))
        y_mov = y_mov/100
        return y_mov
    else:
        return y_prv

def update_freq(x_mov,x_prv):
    print("previous x: "+ str(x_prv))
    print("current x: "+ str(x_mov))
    #minimum movment to change parameter
    if(abs(x_prv-x_mov)>= 5):
        print("changing frequency from "+str(x_prv)+" to"+str(x_mov))
        return x_mov
    else:
        return x_prv
        
        
        
amp = 1 
f = 440 #frequency
start_idx = 0
block = 100 #the smaller the value, less latency (but if too low input underflow)
lat = 0 #latency in seconds
srate = 2048        
        
        
     
        
        
def callback(indata, outdata, frames, time, status):
        if status:
            print(status, file=sys.stderr)
        global start_idx
        t = (start_idx + np.arange(frames)) / srate
        t = t.reshape(-1, 1)
        
        global f
        global amp
        indata = amp * np.sin(2 * np.pi * f * t) #create sine wave
        outdata[:] = indata  
        start_idx += frames          
        
        
        
        
        
        
        
        
        
        
        
        
      
        
with sd.Stream(
                   samplerate=srate, blocksize=block,latency=lat,
                   channels=1, callback=callback ):        
        
    while(True): 

        timer = cv2.getTickCount() 
        # Capture the video frame 
        ret, frame = vid.read() 

        #searches for object in frame
        ret, bbox = tracker.update(frame)

        #if object is found in frame
        if ret:
            x,x_prv, y, y_prv = drawBox(frame,bbox)

            #update parameter values 
            #amp = float(update_amp(y,y_prv))
            f =  float(update_freq(x,x_prv))

            #update synth stream 


















        #if object is not found in frame        
        else:
            cv2.putText(frame,"Lost",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)

        fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer)
        cv2.putText(frame,str(int(fps)),(75,50), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)

        # Display the resulting frame 
        cv2.imshow('frame', frame) 

        # press q to quit
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

            
            
            
            
            
            
            
            
            
            
            
            
            
            

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 


previous x: 337
current x: 337
previous x: 337
current x: 337
previous x: 337
current x: 337
previous x: 337
current x: 336
previous x: 336
current x: 336
previous x: 336
current x: 336
previous x: 336
current x: 337
previous x: 337
current x: 334
previous x: 334
current x: 331
previous x: 331
current x: 326
changing frequency from 331 to326
previous x: 326
current x: 320
changing frequency from 326 to320
previous x: 320
current x: 311
changing frequency from 320 to311
previous x: 311
current x: 289
changing frequency from 311 to289
previous x: 289
current x: 275
changing frequency from 289 to275
previous x: 275
current x: 252
changing frequency from 275 to252
previous x: 252
current x: 232
changing frequency from 252 to232
previous x: 232
current x: 210
changing frequency from 232 to210
previous x: 210
current x: 178
changing frequency from 210 to178
previous x: 178
current x: 159
changing frequency from 178 to159
previous x: 159
current x: 156
previous x: 156
current x: 144
changing 

previous x: 252
current x: 246
changing frequency from 252 to246
previous x: 246
current x: 245
previous x: 245
current x: 232
changing frequency from 245 to232
previous x: 232
current x: 229
previous x: 229
current x: 228
previous x: 228
current x: 227
previous x: 227
current x: 224
previous x: 224
current x: 222
previous x: 222
current x: 220
previous x: 220
current x: 219
previous x: 219
current x: 219
previous x: 219
current x: 218
previous x: 218
current x: 219
previous x: 219
current x: 218


In [2]:

    
    
    
    
def MIDI_to_freq(note):
    exp = (note - 69)/ 12
    freq = 2 ** exp * 440
    return freq

def play_audio(data, sr):
    sd.play(data, sr)
    status = sd.wait()
    
def sinusoid(freq, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data
